In [ ]:
import geopandas
import scipy.io
import numpy as np
import pandas as pd
import xarray as xr
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

In [ ]:
# get geopandas representation of final_biomass_table
# allows us to plot transect data
df = epro_2019.final_biomass_table
gdf = geopandas.GeoDataFrame(df, 
                             geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

# reseting the index so that we can select data based on column name
gdf = gdf.reset_index()

## Generate length-weight and length-age keys for each stratum

In [ ]:
# import class to use it's functions 
# from EchoPro.load_biological_data import LoadBioData

In [ ]:
# length_explode_df = epro_2019.length_df[['Sex', 'Length']].copy()
# # add strata column 
# length_explode_df['Strata'] = length_explode_df.reset_index().apply(lambda x: strata_haul_df.loc[x[0]], 
#                                                                     axis=1).values

# length_explode_df.reset_index(inplace=True)

# length_explode_df.set_index('Strata', inplace=True)

# # add weight column with NaNs
# length_explode_df['Weight'] = np.nan
# length_explode_df = length_explode_df.explode(['Sex', 'Length'])

# length_explode_df = length_explode_df.astype({'Haul': int, 
#                                               'Sex': int, 
#                                               'Length': np.float64, 'Weight': np.float64})

## Calculate the biomass density

In [ ]:
# nasc_df = epro_2019.load_nasc_data()

In [ ]:
# # calculates the interval for the area calculation
# interval = (nasc_df['VL start'].iloc[1:].values - nasc_df['VL start'].iloc[:-1].values)
# last_interval = nasc_df['VL end'].iloc[-1] - nasc_df['VL start'].iloc[-1]

# interval = np.concatenate([interval, np.array([last_interval])])

# median_interval = np.median(interval)

# # remove outliers at the end of the transect
# ind_outliers = np.argwhere(np.abs(interval - median_interval) > 0.05).flatten()
# interval[ind_outliers] = nasc_df['VL end'].values[ind_outliers] - nasc_df['VL start'].values[ind_outliers]

In [ ]:
# bio_dense_df = nasc_df[['Stratum', 'NASC', 'Haul']].copy()
# bio_dense_df['interval'] = interval
# bio_dense_df['n_A'] = nasc_df.apply(lambda x: x.NASC/epro_2019.strata_ds.sig_b.sel(strata=x.Stratum).values, axis=1)
# bio_dense_df['A'] = bio_dense_df['interval']*nasc_df['Spacing']
# bio_dense_df['N_A'] = bio_dense_df['n_A']*bio_dense_df['A']

In [ ]:
# bio_density = bio_dense_df.apply(lambda x: x.N_A*np.dot(len_wgt_ALL_da.sel(strata=x.Stratum), 
#                                           norm_len_key_ALL_da.sel(strata=x.Stratum)), axis=1)

In [ ]:
# bio_density.reset_index().loc[734:740]

In [ ]:
# bio_dense_df.reset_index().loc[734:740]

## Generate keys

In [ ]:
# import class to use it's functions 
from EchoPro.load_stratification_data import LoadStrataData

# get df relating the haul to the stratum
strata_haul_df = epro_2019.strata_df.reset_index()[['Haul', 'strata']].set_index('Haul')

# get the bins for the lengths
bins_len = epro_2019.params['bio_hake_len_bin']

# get the bins for the ages
bins_age = epro_2019.params['bio_hake_age_bin']

# get all specimen data that is necessary for key generation
spec_w_strata = epro_2019.specimen_df.drop('Specimen_Number', axis=1).copy().reset_index()

# add strata column 
spec_w_strata['Strata'] = spec_w_strata.apply(lambda x: strata_haul_df.loc[x[0]], 
                                              axis=1).values

spec_w_strata.set_index('Strata', inplace=True)

spec_w_strata = spec_w_strata[(spec_w_strata['Age'] != 0) & (spec_w_strata['Sex'] != 3)].copy()
age_len_key_da, age_len_key_wgt_da, age_len_key_norm_da = LoadStrataData.get_age_key_das(LoadStrataData, 
                                                                                         spec_w_strata, 
                                                                                         bins_len, bins_age)

In [ ]:
age_len_key_norm_da.sel(strata=1).values[:, :2]

In [ ]:
# def get_weight_key_das(spec_w_strata, bins_len, reg_w0, reg_p):
    
#     unique_strata = np.sort(spec_w_strata.index.unique())
    
#     len_wgt_key = np.empty((unique_strata.shape[0], bins_len.shape[0]))
#     len_wgt_key[:, :] = np.nan

#     len_key = np.empty((unique_strata.shape[0], bins_len.shape[0]))
#     len_key[:, :] = np.nan
    
#     len_key_norm = np.empty((unique_strata.shape[0], bins_len.shape[0]))
#     len_key_norm[:, :] = np.nan

#     stratum_i = 0
#     for stratum in unique_strata:
        
#         l_w_k, l_k, l_k_n = LoadStrataData.generate_length_val_key(LoadStrataData, bins_len, reg_w0, reg_p,
#                                                len_name='Length', 
#                                                val_name='Weight', 
#                                                df=spec_w_strata.loc[stratum])
        
#         len_wgt_key[stratum_i, :] = l_w_k 
#         len_key[stratum_i, :] = l_k 
#         len_key_norm[stratum_i, :] = l_k_n
        
#         stratum_i += 1
        
        
#     len_wgt_key_da = xr.DataArray(data=len_wgt_key, 
#                                   coords={'strata': unique_strata, 'len_bins': bins_len})
    
#     len_key_da = xr.DataArray(data=len_key, 
#                                   coords={'strata': unique_strata, 'len_bins': bins_len})
    
#     len_key_norm_da = xr.DataArray(data=len_key_norm, 
#                                   coords={'strata': unique_strata, 'len_bins': bins_len})
    
#     return len_wgt_key_da, len_key_da, len_key_norm_da
    

In [ ]:
# get all specimen data that is necessary for key generation
spec_w_strata = epro_2019.specimen_df.drop('Specimen_Number', axis=1).copy().reset_index()

# add strata column 
spec_w_strata['Strata'] = spec_w_strata.apply(lambda x: strata_haul_df.loc[x[0]], 
                                              axis=1).values

spec_w_strata.set_index('Strata', inplace=True)

reg_w0, reg_p = LoadStrataData.get_length_val_reg_vals(len_name='Length', val_name="Weight", df=spec_w_strata)

spec_w_strata = spec_w_strata.copy() #[(spec_w_strata['Age'] != 0)].copy()
len_wgt_key_da, len_key_da, len_key_norm_da = LoadStrataData.get_weight_key_das(LoadStrataData, 
                                                                                spec_w_strata, 
                                                                                bins_len, reg_w0, reg_p)

In [ ]:
len_wgt_key_da.sel(strata=1)

In [ ]:
# %%time
# lat_INPFC = [np.NINF, 36, 40.5, 43.000, 45.7667, 48.5, 55.0000]  # INPFC
# epro_2019.run_cv_analysis(lat_INPFC, kriged_data=False)

In [ ]:
# CV_JH_mean = 0.13377365505687697
# CPU times: user 2min 31s, sys: 312 ms, total: 2min 31s
# Wall time: 2min 31s